In [1]:
import torch
import torch.nn as nn
device = "cuda"
device = torch.device(device)
print(device)

c:\Users\mrcgg\anaconda3\envs\ML-pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# add mode 2^32
def add(a, b):
    return (a + b) % 2**32

In [3]:
import torch.nn.functional as F
noise = 0.0
noise_n = 1

train_data = []

# every rotation is possible using only k=1

while len(train_data) < 100000:
    i = torch.randint(0, 2**32, (1,)).item()
    j = torch.randint(0, 2**32, (1,)).item()
    for _ in range(noise_n):
        a = torch.tensor([float(a) for a in bin(i)[2:].rjust(32,"0")], dtype=torch.float32)
        # a = torch.tensor([float(i)], dtype=torch.float32)
        a = a.to(device)

        b = torch.tensor([float(k) for k in bin(j)[2:].rjust(32,"0")], dtype=torch.float32)
        # b = torch.tensor([float(j)], dtype=torch.float32)
        b = b.to(device)

        res = torch.tensor([float(k) for k in bin(add(i,j))[2:].rjust(32, "0")], dtype=torch.float32)
        # res = torch.tensor([float(add(i,j))], dtype=torch.float32)
        res = res.to(device)

        # inp = torch.stack((a,b), dim=0).view(32*2)
        # interleave a and b
        inp = torch.zeros(32*2, dtype=torch.float32)
        inp[0::2] = a
        inp[1::2] = b
        # reverse the order of the bits
        inp = inp.view(32,2)
        inp = inp.flip(0)
        inp = inp.to(device)
        train_data.append((inp,res))

In [4]:
# for x,y in train_data:
#     if x.shape != torch.Size([32+5]):
#         print(x.shape)
#         print(x,y)
#         break
#     if y.shape != torch.Size([32]):
#         print(y.shape)
#         print(x, y)
#         break
train_data[0]

(tensor([[0., 1.],
         [1., 0.],
         [1., 1.],
         [0., 0.],
         [0., 1.],
         [0., 0.],
         [0., 0.],
         [1., 0.],
         [1., 0.],
         [0., 0.],
         [0., 0.],
         [0., 1.],
         [1., 1.],
         [1., 0.],
         [0., 0.],
         [1., 1.],
         [1., 1.],
         [0., 1.],
         [0., 0.],
         [1., 0.],
         [1., 1.],
         [1., 0.],
         [0., 0.],
         [1., 1.],
         [1., 0.],
         [0., 0.],
         [0., 1.],
         [1., 0.],
         [0., 0.],
         [0., 1.],
         [0., 0.],
         [1., 1.]], device='cuda:0'),
 tensor([0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
         0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1.],
        device='cuda:0'))

In [5]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

def get_loaders(train_data, device=device):
    test_size = 0.05
    valid_size = 0.05
    batch_size = 500
    num_workers = 0

    #cuda or cpu
    device = torch.device(device)

    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(test_size * num_train))
    split2 = int(np.floor((valid_size+test_size) * num_train))
    train_idx, valid_idx, test_idx = indices[split2:], indices[split:split2], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    # prepare data loaders
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
    valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=test_sampler, num_workers=num_workers)
    return train_loader, valid_loader, test_loader

train_loader, valid_loader, test_loader = get_loaders(train_data)

In [6]:
import numpy as np
def noise_to_int(bits):
    bits = [round(float(b)) for b in bits]
    bits = "".join([str(b) if b in [0,1] else "0" if b<1/10**5 else "1" for b in bits])
    return int(bits,2)

def lin_to_tuple_32_32(t):
    a = (t[:32],t[32:])
    return tuple([noise_to_int(b) for b in a])

def lin_to_list(t):
    res = []
    for i in range(len(t)//8):
        res += [noise_to_int(t[i*8:i*8+8])]
    return res

In [7]:
X, Y = next(iter(train_loader))

for x,y in zip(X,Y):
    a = torch.zeros(32)
    b = torch.zeros(32)
    for i in range(32):
        a[31-i] = x[i][0]
        b[31-i] = x[i][1]
    a = noise_to_int(a)
    b = noise_to_int(b)
    res = noise_to_int(y)
    # print(a,b)
    # print(x)
    # print(y)
    b1 = add(a,b)
    # print(b1, res)
    assert res == b1
print("ok")

ok


In [8]:
import torch.nn.functional as F

# use RNN to learn the add function
class Add(nn.Module):
    def __init__(self):     
        super(Add, self).__init__()
        self.hidden_size = 2
        self.num_layer = 2
        self.input_size = 2
        self.seq_len = 32
        self.rnn = nn.LSTM(self.input_size, self.hidden_size, self.num_layer, batch_first=True, dtype=torch.float32)
        self.fc = nn.Linear(self.hidden_size * self.seq_len, self.seq_len, dtype=torch.float32)

    def forward(self, x):
        batch_size = x.size(0)
        x, h = self.rnn(x)
        # print(x.shape)
        x = x.reshape(batch_size, self.hidden_size * self.seq_len)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

In [9]:
def train(save_file, model, criterion, train_loader, valid_loader, optimizer=None, n_epochs = 100000, f=noise_to_int, lrate=0.005):
    # number of epochs to train the model

    if optimizer is None:
        # specify optimizer (stochastic gradient descent) and learning rate = 0.001
        optimizer = torch.optim.Adam(model.parameters(), lr=lrate)#, weight_decay=0.00000001)

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf # set initial "min" to infinity
    
    for epoch in range(n_epochs):
        # monitor training loss
        train_loss = 0.0
        valid_loss = 0.0
        results = 0
        results_n = 0
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        i=0
        for X, target in train_loader:
            i+=1
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            target = target.to(device)
            output = model(X)
            # calculate the loss
            # print(output)
            # print(target)
            loss = criterion(output, target) #
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss += loss.item()*X.size(0)
            if epoch%100 == 0:
                for x,y in zip(output,target):
                    # print(x.cpu().detach().numpy(),y)
                    a = f(x.cpu().detach().numpy())
                    # a = int(x[0])
                    b = f(y.cpu().detach().numpy())
                    # b = int(y[0])
                    # a = noise_to_int(x)
                    # b = noise_to_int(y)
                    
                    
                    # print(a,b)
                    # print(float(x[0]),float(y[0]))
                    if a==b:
                        
                        results +=1
                    results_n+=1
        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        for X, target in valid_loader:
        
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # target = target.to(device)
            # calculate the loss
            loss = criterion(output, target)
            # update running validation loss
            valid_loss += loss.item()*X.size(0)
            

        # print training/validation statistics
        # calculate average loss over an epoch
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)

        print('Epoch: {} \tTraining Loss: {:.14f} \tValidation Loss: {:.14f}'.format(
            epoch+1,
            train_loss,
            valid_loss
            ))

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.14f} --> {:.14f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
            torch.save(model.state_dict(), save_file)
            valid_loss_min = valid_loss
            if train_loss <= 0.0000000001:
                print("stop: loss <= 0.00000")
                return
            else:
                print(" loss >= 0.00000")
        
        if results_n != 0 :
            print(f"{results/results_n=}")
            print(f"{results}")
            # if results == results_n and valid_loss <= valid_loss_min:
            #     print("stop: no errors")
            #     return
        

In [10]:
model = Add().to(device)
criterion = nn.BCELoss()
train("add.pt", model, criterion, train_loader, valid_loader, f=noise_to_int, lrate=0.01)

Epoch: 1 	Training Loss: 0.61023167490959 	Validation Loss: 0.02737148642540
Validation loss decreased (inf --> 0.02737148642540).  Saving model ...
 loss >= 0.00000
results/results_n=0.0
0
Epoch: 2 	Training Loss: 0.23317887801677 	Validation Loss: 0.00633649788797
Validation loss decreased (0.02737148642540 --> 0.00633649788797).  Saving model ...
 loss >= 0.00000
Epoch: 3 	Training Loss: 0.06472138375044 	Validation Loss: 0.00177221089602
Validation loss decreased (0.00633649788797 --> 0.00177221089602).  Saving model ...
 loss >= 0.00000
Epoch: 4 	Training Loss: 0.02025296614040 	Validation Loss: 0.00072368561756
Validation loss decreased (0.00177221089602 --> 0.00072368561756).  Saving model ...
 loss >= 0.00000
Epoch: 5 	Training Loss: 0.00963618354406 	Validation Loss: 0.00040468190797
Validation loss decreased (0.00072368561756 --> 0.00040468190797).  Saving model ...
 loss >= 0.00000
Epoch: 6 	Training Loss: 0.00583787360927 	Validation Loss: 0.00026568951318
Validation loss d

KeyboardInterrupt: 

In [11]:
model = Add().to(device)
model.load_state_dict(torch.load("add.pt"))

<All keys matched successfully>

In [12]:
X, Y = next(iter(test_loader))
O = model(X)
print(Y[1], O[1])

tensor([1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.],
       device='cuda:0') tensor([1.0000e+00, 1.0000e+00, 3.6383e-10, 1.5164e-11, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 7.4644e-11, 1.0000e+00, 1.0000e+00, 3.9853e-11, 1.1535e-10,
        8.6950e-11, 1.0000e+00, 3.1158e-10, 1.0000e+00, 8.1699e-11, 2.5697e-10,
        1.0000e+00, 1.3292e-11, 1.8079e-11, 6.9031e-11, 3.8012e-10, 1.0000e+00,
        1.4691e-10, 2.0301e-10, 1.0000e+00, 6.0128e-11, 1.0000e+00, 6.4623e-11,
        1.0000e+00, 5.9685e-11], device='cuda:0', grad_fn=<SelectBackward0>)


In [13]:
model.eval()

results = 0
results_n = 0

for X,Y in train_loader:
    # print(X,Y)
    with torch.no_grad():
        O = model(X)
    for x,y in zip(O,Y):
        a = noise_to_int(x.cpu().detach().numpy())
        b = noise_to_int(y.cpu().detach().numpy())
        # b = int(y[0]) pos_to_int
        # b = noise_to_int(y)
        # print(x,y)
        # print(a,b)
        if a==b:
            
            results +=1
        results_n +=1
print(f"{results/results_n=}")

results/results_n=1.0


In [27]:
model.eval()

results = 0
results_n = 0
noise = 0.01
for i in range(10000):
    a = torch.randint(0, 2**32, (1,)).item()
    b = torch.randint(0, 2**32, (1,)).item()
    A = torch.tensor([float(a) for a in bin(a)[2:].rjust(32,"0")], dtype=torch.float32)
    A += (torch.FloatTensor(32).uniform_(-noise, +noise))
    B = torch.tensor([float(a) for a in bin(b)[2:].rjust(32,"0")], dtype=torch.float32)
    B += (torch.FloatTensor(32).uniform_(-noise, +noise))
    
    X = torch.zeros(32*2, dtype=torch.float32)
    X[0::2] = A
    X[1::2] = B
    X = X.view(32,2)
    X = X.flip(0)
    X = X.to(device)
    X = X.unsqueeze(0)

    Y = torch.tensor([float(a) for a in bin(add(a,b))[2:].rjust(32,"0")], dtype=torch.float32).to(device)
    Y = Y.unsqueeze(0)
    with torch.no_grad():
        O = model(X)
    for x,y in zip(O,Y):
        a = noise_to_int(x.cpu().detach().numpy())
        b = noise_to_int(y.cpu().detach().numpy())
        # b = int(y[0]) pos_to_int
        # b = noise_to_int(y)
        # print(x,y)
        # print(a,b)
        if a==b:
            
            results +=1
        else:
            print(a,b)
        results_n +=1
print(f"{results/results_n=}")

results/results_n=1.0


In [18]:
for layer in model.children():
    if isinstance(layer, nn.Linear):
        print("linear")
        print(layer.weight.shape)
        print(layer.bias.shape)
        print(layer.weight)
        print(layer.bias)
    if isinstance(layer, nn.LSTM):
        print("lstm")
        print(layer.weight_ih_l0.shape)
        print(layer.weight_hh_l0.shape)
        print(layer.bias_ih_l0.shape)
        print(layer.bias_hh_l0.shape)
        print(layer.weight_ih_l0)
        print(layer.weight_hh_l0)
        print(layer.bias_ih_l0)
        print(layer.bias_hh_l0)
        print(layer.hidden_size)
        print(layer.input_size)
        print(layer.num_layers)
        print(layer.bias)
        print(layer.dropout)
        print(layer.bidirectional)
        print(layer.batch_first)
        print(layer.proj_size)
        print(layer)
    print()


lstm
torch.Size([8, 2])
torch.Size([8, 2])
torch.Size([8])
torch.Size([8])
Parameter containing:
tensor([[-6.2521,  5.9953],
        [ 4.6506,  1.7229],
        [-1.6022, -2.4458],
        [ 2.7108,  4.5105],
        [-6.3193,  2.4689],
        [ 0.5226, -7.1532],
        [-5.5024,  5.5650],
        [ 4.7941,  5.5716]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 3.0304,  8.0028],
        [-0.1866, -3.8193],
        [-1.3315,  6.0119],
        [-1.7743, -0.6462],
        [ 0.2903,  3.2302],
        [ 0.3963,  4.5638],
        [ 0.8516,  8.9225],
        [ 0.1657,  1.7805]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 1.6164, -2.5298, -0.7971, -1.9898, -1.2576,  0.0660,  1.2347, -1.5695],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 1.5969, -3.2419, -0.4246, -2.5121, -0.9013, -0.5101,  1.7107, -1.5696],
       device='cuda:0', requires_grad=True)
2
2
2
True
0.0
False
True
0
LSTM(2, 2, num_layers=2, batch_firs

In [ ]:
import torch
state = torch.load("xor.pt")

for w in state["body.0.weight"]:
    for i in range(len(w)):
        print(w)
        if w[i] > 0:
            w[i] = 100.0
        else:
            w[i] = -100.0

for i in range(len(state["body.0.bias"])):
    state["body.0.bias"][i] = -50.0
    print(state["body.0.bias"][i])

for w in state["body.2.weight"]:
    for i in range(len(w)):
        print(w)
        if w[i] > 0:
            w[i] = 200.0
        else:
            w[i] = 0.0

for i in range(len(state["body.2.bias"])):
    state["body.2.bias"][i] = -100.0
    print(state["body.2.bias"][i])
torch.save(state,"xor.pt")

tensor([-10.,  10.], device='cuda:0')
tensor([-100.,   10.], device='cuda:0')
tensor([ 10., -10.], device='cuda:0')
tensor([100., -10.], device='cuda:0')
tensor(-50., device='cuda:0')
tensor(-50., device='cuda:0')
tensor([20., 20.], device='cuda:0')
tensor([200.,  20.], device='cuda:0')
tensor(-100., device='cuda:0')
